# Feature Selection and Classification
## Purpose
This notebook performs training and internal validation for the LVSD etiology classifier.

## Prerequisites
- Extracted wall thicknesses and chamber volumes as computed by `ukbb_cardiac`
- Radiomic features extracted using `pyradiomics` from segmented T1 map images (see `ValidateMasksExtractFeatures.ipynb`)
- Heart failure etiology classifications (see `HFClassification.py`)
    - Include covariates extracted from UK Biobank's phenotype tables, as named in [their public Showcase documentation](https://biobank.ndph.ox.ac.uk/showcase/)
- Python environment (3.9.x) with dependencies specified in `requirements.txt`

## Local Dependencies
- `FeatureSelector.py`: Pipeline module responsible for selecting a minimum number of features through RFECV and constraining to a maximum number of features through heirarchical clustering
- `HFClassification.py`: Enumeration of etiologic diagnoses considered in this study.
## Import packages and data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import RocCurveDisplay, f1_score, confusion_matrix
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from MLstatkit.stats import Bootstrapping
from HFClassification import HFClassification
from FeatureSelector import FeatureSelector
shmolli_extracted_features_path = ''#path to CSV contianing extracted radiomic features from T1 map, listed by EID. See ValidateMasksExtractFeatures.ipynb.
excluded_by_bad_mri_path = ''#list any MRI to exclude over segmentation quality
phenotypes_path = ''#table of participant EIDs and their corresponding HFClassification diagnoses
short_axis_path = ''#results generated by ukbb_cardiac segmentation

shmolli_extracted_features = pd.read_csv(shmolli_extracted_features_path , low_memory=False)
excluded_by_bad_mri = pd.read_csv(excluded_by_bad_mri_path)
phenotypes = pd.read_csv(phenotypes_path)
short_axis = pd.read_csv(short_axis_path)

## Preprocessing
- Records of blood pressures in UKBB are exploded across 16 tabular columns. We just want one reading for systolic BP and one reading for diastolic BP as aquired at the start of an imaging appointment. We will prefer manual readings over automated readings if both are available, due to higher accuracy
- Drop patients who do not have valid MRI or whose MRI were deemed of insufficient segmentation quality on review. Select the earliest avialable pair of MRI from each patient.
- Merge tables imported above together.

In [ ]:
phenotypes['systolic_bp'] = phenotypes.p93_i2_a0.fillna(phenotypes.p4080_i2_a0).fillna(phenotypes.p93_i3_a0).fillna(phenotypes.p4080_i3_a0)
phenotypes['diastolic_bp'] = phenotypes.p94_i2_a0.fillna(phenotypes.p4079_i2_a0).fillna(phenotypes.p94_i3_a0).fillna(phenotypes.p4079_i3_a0)

quality_or_segmentation_failures =excluded_by_bad_mri[~excluded_by_bad_mri.passable_shmolli_quality].mask_path.str.replace('_myocardium.png' , '.dcm').reset_index(drop=True)
shmolli_extracted_features.drop(columns = quality_or_segmentation_failures , inplace=True)

shmolli_extracted_T = shmolli_extracted_features.T
shmolli_extracted_T.columns = shmolli_extracted_T.iloc[0]
shmolli_extracted_T = shmolli_extracted_T.reset_index().sort_values(by = 'index')
shmolli_extracted_T = shmolli_extracted_T.drop(index = shmolli_extracted_T.loc[shmolli_extracted_T['index'] == 'Unnamed: 0'].index)
shmolli_extracted_T['eid'] = shmolli_extracted_T['index'].apply(lambda s : int(s.split('/')[-2]))
shmolli_extracted_T.drop_duplicates(subset = 'eid' , keep = 'first' , inplace = True)
shmolli_extracted_T.drop(columns = [c for c in shmolli_extracted_T.columns if 'diagnostic' in c])

print(f'Number of patients in eligible patient phenotype CSV: {phenotypes.shape[0]}')
print(f'...who also had a ShMOLLI MRI of passable quality which was segmented successfully: {shmolli_extracted_T.shape[0]}')

short_axis.sort_values(by = ['eid' , 'cMRI_date'] , inplace = True)
short_axis.drop_duplicates(subset = ['eid','cMRI_date'] , keep = 'first' , inplace = True)
short_axis.drop_duplicates(subset = ['eid'] , keep = 'first' , inplace = True)
short_axis = short_axis.drop(columns = ['sex','dob_year','dob_month','height_cm','weight_kg','ethnicity','cMRI_date','hf_date','hf_codes'])

phenotypes['HFClassification'] = phenotypes.class_int.apply(lambda i : HFClassification(i))
phenotypes['class_vector'] = phenotypes.HFClassification.apply(lambda h : h.__vector__())
full_dataset = (phenotypes.merge(shmolli_extracted_T , how = 'inner' , on = 'eid')).merge(short_axis , how = 'inner' , on = 'eid')

## Identify Covariates and Features; Ensure All are Numeric

In [ ]:
covariates = [
    'p21003_i2',#age at first imaging visit
    'p21001_i2',#bmi at first imaging visit
    'p30030_i2',#hematocrit at imaging visit
    'p30030_i0',#hematocrit as measured at first Biobank visit
    'p30030_i1',#hematocrit as measured at second Biobank visit
    'systolic_bp',
    'diastolic_bp',
    'p31',#sex
]

shmolli_features = [i for i in shmolli_extracted_T.columns if i != 'index' and 'diagnostics' not in i and i != 'eid']
short_axis_features = [i for i in short_axis.columns if i != 'index' and i != 'eid']
all_features = covariates + short_axis_features + shmolli_features

#Make sure all of our fields are numeric and not entirely empty
full_dataset[['p53_i0','p53_i1','imaging_date','p53_i3']] = \
    full_dataset[['p53_i0','p53_i1','imaging_date','p53_i3']].apply(lambda t : pd.to_datetime(t).astype(np.int64) if ~pd.isna(t).any() else None)
full_dataset[all_features] = full_dataset[all_features].apply(pd.to_numeric)
covariates = [c for c in covariates if ~pd.isna(full_dataset[c]).all()]

# Chose Egiologies for your Classifier to Identify
- List the `HFClassification`s which you wish your model to differentiate.
    - If you have an intuition of good chain ordering for your diagnostic labels, list your labels here in preferred sort order. This order will inform classifier chain ordering. (The ordering below is based on the clinical definition as ICM and NICM as distinct etiologies, with further subtypes representing subtypes of NICM.)
- These will be assigned binarized labels, with multi-output encoding.
- Everything else in the training set will be assigned $y=\boldsymbol{0}$

In [ ]:
chain_ordered_classifications = [HFClassification.ICM , HFClassification.ARRHYTHMIA , HFClassification.HYPERTENSIVE]

labels_to_binarize = np.array(chain_ordered_classifications).reshape(1,-1)
Y = (np.expand_dims(full_dataset.HFClassification , axis = 1) & labels_to_binarize).astype(bool).astype(np.uint8)

has_classification = Y.sum(axis = 1) > 0
full_dataset = full_dataset[has_classification]
Y = Y[has_classification]

print('Encoding of chosen classifications:')
print(labels_to_binarize.squeeze())
unique_rows , counts = np.unique(Y , axis = 0 , return_counts = True)
print('Number of instances of each encoding:')
for row , count in zip(unique_rows , counts):
    print(f'Encoding {row} {" | ".join([labels_to_binarize[0,i].__str__() for i in np.where(row)[0]])}: {count} instances')

## Train-Test Split
- We'll split our training from our test data now.
- Test data will be reserved for internal validation of our finalized classifier.
- Stratified K-folding will be used to split training and validation sets as we both select features and train our classifier below.

In [ ]:
test_size = 0.2#percent test for your train test split
random_state = 888#will be used for all random number generation
X_train , X_test , Y_train , Y_test = train_test_split(
    full_dataset[all_features] ,
    Y ,
    test_size = test_size ,
    stratify = Y ,
    random_state = random_state
)

print(f'Training X shape: {X_train.shape}')
print('Training class counts:')
unique_rows , counts = np.unique(Y_train , axis = 0 , return_counts = True)
for row , count in zip(unique_rows , counts):
    print(f'Encoding {row} {" | ".join([labels_to_binarize[0,i].__str__() for i in np.where(row)[0]])}: {count} instances')
    
print(f'Test X shape: {X_test.shape}')
print('Test class counts:')
unique_rows , counts = np.unique(Y_test , axis = 0 , return_counts = True)
for row , count in zip(unique_rows , counts):
    print(f'Encoding {row} {" | ".join([labels_to_binarize[0,i].__str__() for i in np.where(row)[0]])}: {count} instances')

## Train Pipeline on Train Data

In [ ]:
# hyperparameters for preprocessing and feature selection
min_features_to_select = 20#min number of features to pass from feature selection
n_jobs = -1#set to n cpu cores
cross_correlation_threshold = 0.8#for heirarchical clustering
stratified_k_fold_n_splits = 5
RFECV_threshold = 1
iterative_imputer_max_iter = 100000
scoring_metric = 'roc_auc' #we're decomposing this problem into m binary classification problems, so no need for a multilabel scoring metric

#define pipeline components
#imputer acting only on the covariates
imputer = IterativeImputer(max_iter=iterative_imputer_max_iter, random_state=random_state)
column_subset_imputer = ColumnTransformer(
    transformers=[
        ('imputer', imputer , covariates),
    ],
    remainder='passthrough'
)
#standard scaler
scaler = StandardScaler()

#rfecv_model = RandomForestClassifier(class_weight='balanced',random_state=random_state,n_jobs=n_jobs)
rfecv_model = LogisticRegression(class_weight='balanced',C=0.3)

feature_selector = FeatureSelector(
    estimator = rfecv_model ,
    step = 1 ,
    cv = StratifiedKFold(n_splits = stratified_k_fold_n_splits) ,
    scoring = scoring_metric,
    min_features_to_select=min_features_to_select ,
    n_jobs = n_jobs
)
#calibrated classifier; logistic regression with balanced class weight chosen as the best option from a grid search
#on ICM and ARRHYTHMIA tasks using this pipeline
base_model = CalibratedClassifierCV(LogisticRegression(class_weight='balanced') , cv = StratifiedKFold(n_splits = stratified_k_fold_n_splits))
binary_pipeline = Pipeline(steps=[
    ('feature_selector' , feature_selector),
    ('classifier', base_model)])

#build a classifier chain
classifier_chain = ClassifierChain(binary_pipeline ,
                        order=None,
                        cv = None,
                        chain_method='predict_proba',
                        random_state=random_state)

#construct pipeline
myfit = Pipeline(steps=[
    ('imputer', column_subset_imputer),
    ('scaler', scaler),
    ('classifier_chain', classifier_chain),
])

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    myfit.fit(X_train , Y_train)
display(myfit)

## Determine Optimal Threshold by Diagnosis
- 0.5 may not represent an accurate significance cutoff for all etiologies due to our high class imbalance.
- To address this, compute a significance cutoff which maximized model F1 statistic performance in the training data set.
- Optimized thresholds as used in poster presentation:
    - ICM: 0.43
    - ARRHYTHMIA: 0.31
    - Hypertensive: 0.12

In [ ]:
def get_optimal_threshold(y_test:np.ndarray , y_prob:np.ndarray) -> int:
    """
    `get_optimal_threshold`
    Maximize the F1 by chosing the best cutoff on an ROC curve.
    Arguments:
    y_test: array of binary true labels (m observations * 1 column)
    y_prob: array of predicted probabilities (m observations * 1 column)
    Returns:
    Significance cutoff which maximizes F1, balancing precision and recall
    """
    best_threshold=-1
    best_f1=-1
    for threshold in np.arange(0.0, 1.0, 0.01):
        y_pred = (y_prob >= threshold).astype(int)
        f1 = f1_score(y_test, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
    return best_threshold

Y_fit = myfit.predict_proba(X_train)
optimal_thresholds = [None for i in range(0 , Y_fit.shape[1])]
for i in range(0 , Y_fit.shape[1]):
    optimal_thresholds[i] = get_optimal_threshold(
        np.array(Y_train[:,i]) ,
        np.array(Y_fit[:,i])
    )
print('Optimized thresholds:')
display(pd.DataFrame({'Class':chain_ordered_classifications , 'Threshold':optimal_thresholds}))


## Visualize and Save Feature Importances
- We'll visualize the sources of our features and their importance rankings.

In [ ]:
sns.set_theme()
sns.set_context('talk')
plt.figure(figsize = (15,6))
colorblind_palette = sns.color_palette("Set2")
color_dict = {
    "Cine": colorblind_palette[0],
    "Covariate": colorblind_palette[1],
    "T1 Map": colorblind_palette[2],
    "Other Class" : colorblind_palette[3],
}
legend_handles = [
    mpatches.Patch(color=color_dict["Cine"], label='Cine'),
    mpatches.Patch(color=color_dict["Covariate"], label='Covariate'),
    mpatches.Patch(color=color_dict["T1 Map"], label='T1 Map'),
    mpatches.Patch(color=color_dict["Other Class"], label="Other Etiology\nDecision")
]

features = myfit.named_steps['imputer'].get_feature_names_out()
for i , p in enumerate(myfit.named_steps['classifier_chain'].estimators_):
    if i > 0:
        features = np.append(features , f'{chain_ordered_classifications[i-1]} Decision')
    myclass = labels_to_binarize[0,i].__str__()
    chosen = p.named_steps['feature_selector'].get_support()
    ranks = p.named_steps['feature_selector'].ranking_
    rankings_df = pd.DataFrame({'feature':features,'was_chosen':chosen,'rank':ranks})
    rankings_df['Source'] = rankings_df['feature'].apply(lambda s : "Cine" if '(' in s else "Covariate" if 'imputer' in s else 'Other Class' if 'Decision' in s else "T1 Map")
    rankings_df['feature'] = rankings_df['feature'].apply(lambda s : s.split('__')[-1].replace('_' , ' ').split('( ')[0])
    rankings_df['color'] = rankings_df['Source'].apply(lambda s : color_dict[s])
    rankings_df.sort_values(by = 'rank' , inplace=True,ascending=False)
    plt.subplot(i//3+1,3,i+1)
    plt.axis('equal')
    plt.barh(y = rankings_df.loc[rankings_df.was_chosen , 'feature'] , width = rankings_df.loc[rankings_df.was_chosen , 'rank'], color = rankings_df.loc[rankings_df.was_chosen , 'color'])
    plt.title(myclass)
    plt.xlabel('Importance Rank')
    plt.ylabel(f'{rankings_df.was_chosen.sum()} Features Selected from {rankings_df.shape[0]}')
    plt.xlim(0,20)
    if i == 1:
        plt.legend(handles=legend_handles, title='Feature Source', loc='best')
    plt.yticks([])
    plt.tight_layout()
    rankings_df.to_csv(f'{myclass}_features.csv')
plt.savefig('feature_importance.svg',format='svg')

## Helper Code to Plot Classifier Performance

In [ ]:
def plot_model_fit(y_test_in , y_pred_in , classes:list , title:str) -> None:
    """
    `plot_model_fit`
    Draws ROC curves and calibration curves for each classification, given classifier results.
    Arguments:
    y_test_in: The ground-truth labels for y_test
    y_pred_in: The predicted labels for x_test from a classifier.
    classes: Names with which to label each class
    title: Title for the subplots
    """
    sns.set_theme(style='darkgrid', palette='colorblind', context='talk')

    y_test = np.array(y_test_in)
    y_pred = np.array(y_pred_in)
    fig , [ax1 , ax2] = plt.subplots(1,2,figsize=(12,8))
    i = 0
    for class_of_interest , mycolor in zip(classes ,  sns.color_palette('colorblind')[0:len(classes)]):
        plt.subplot(1,2,1)
        my_display = RocCurveDisplay.from_predictions(
            y_test[:,i],
            y_pred[:,i],
            ax = ax1,
            name = class_of_interest,
            color = mycolor,
        );
        ax1 = plt.gca()
        ax1.set(
            xlabel="False Positive Rate",
            ylabel="True Positive Rate",
            title=f"Receiver Operating Characteristic:\n{title}",
        );
        ax1.plot([0, 1], [0, 1], linestyle='--', color='#444455')
        ax1.set_aspect('equal', adjustable='box')
        prob_true , prob_pred = calibration_curve(
            y_test[:,i],
            y_pred[:,i],
            n_bins=4,
        )
        sort_key = np.argsort(prob_true)
        plt.subplot(1,2,2)
        ax2 = plt.gca()
        plt.plot(
            prob_true[sort_key] ,
            prob_pred[sort_key] ,
            linestyle = '-' ,
            marker = '.',
            color = mycolor
        )
        plt.xlim(0,1)
        plt.ylim(0,1)
        ax2.plot([0, 1], [0, 1], linestyle='--', color='#444455')
        ax2.set_title(f'Calibration Curve:\n{title}')
        ax2.set_xlabel('Predicted Probability')
        ax2.set_ylabel('Observed Probability')
        ax2.set_aspect('equal', adjustable='box')
        i+=1
    plt.tight_layout()

## Internally validate the classifier; plot results

In [ ]:
Y_score = np.array(myfit.predict_proba(X_test))

Y_score_df = pd.DataFrame(Y_score ,#transpose to get the probability of each class (col) being true for each observation (row)
                          index=X_test.index,
                          columns=labels_to_binarize.squeeze())
plot_model_fit(Y_test ,
               Y_score_df ,
               labels_to_binarize.squeeze(),
               title = 'Internal Validation')
plt.savefig('ROC.svg',format='svg')

## Plot Confusion Matrix for this Internal Validation Process

In [ ]:
sns.set_theme(style='dark', palette='colorblind', context='poster')

Y_score_list = pd.DataFrame(myfit.predict(X_test) , columns = Y_score_df.columns).apply(lambda h : h.__str__()).tolist()
Y_test_df = pd.DataFrame(Y_test , columns = Y_score_df.columns)

m =  Y_score_df.shape[1]
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
for etiology , threshold , i in zip(Y_score_df.columns , optimal_thresholds , range(0 , m)):
    plt.subplot(np.ceil(m/3).astype(int),3,i+1)
    
    mtx = pd.DataFrame(confusion_matrix(
            Y_test_df[etiology],
            Y_score_df[etiology] > optimal_thresholds[i],
            normalize='true',
    ))
    mtx.index = ['Patient (-)' , 'Patient (+)']
    mtx.columns = ['Model (-)' , 'Model (+)']
    cmap = LinearSegmentedColormap.from_list("custom_cmap", ["white", sns.color_palette('colorblind')[i]])
    sns.heatmap(mtx ,
            cmap = cmap,
            vmin = 0,
            vmax = 1,
            annot=True,
            cbar=False
        )
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title(etiology)
plt.savefig('confusion.svg',format='svg')

## Calculate Confidence Intervals for AUCs
- (We'll compute both a micro-averaged AUC and individual AUCs for each etiology)

In [ ]:
trueval = Y_test.ravel()
prob_A = Y_score.ravel()
stats = pd.DataFrame({'statistic':[] , 'expected':[] , 'lower bound':[] , 'upper bound':[]})
for stat in ['roc_auc']:
    e , lower , upper = Bootstrapping(trueval , prob_A , stat, average='micro')
    stats.loc[len(stats)] = {'statistic':stat , 'expected':e , 'lower bound':lower , 'upper bound':upper}
print('Micro-Averaged AUC')
display(stats)

for col in Y_score_df.columns:
    trueval = np.array(Y_test_df[col].tolist())
    prob_A = np.array(Y_score_df[col].tolist())
    stats = pd.DataFrame({'statistic':[] , 'expected':[] , 'lower bound':[] , 'upper bound':[]})
    print(f'AUC for {col}')
    for stat in ['roc_auc']:
        e , lower , upper = Bootstrapping(trueval , prob_A , stat, average='micro')
        stats.loc[len(stats)] = {'statistic':stat , 'expected':e , 'lower bound':lower , 'upper bound':upper}
        display(stats)

## Save the Model We've Trained
- Trained model as used in poster presentation is included in this repository.

In [ ]:
from joblib import dump
dump(myfit, 'Multiclass_b_chain_v1.joblib')